In [1]:
# Importing Libraries 
import pandas as pd
import psycopg2
import awswrangler as wr
import extract_data
import transformation
import validation
import load
dir(validation)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'pa',
 'validate_dataframe']

EXTRACTION

In [2]:

# File Paths
csv_file_path = r'C:\Users\Kartikay\Downloads\MOCK_DATA_CSV.csv'
json_path = r'C:\Users\Kartikay\Downloads\mock_data_json.json'

# Calling the function in extract_data.py to extract data from different sources
df_sql = extract_data.extract_sql(
    dbname='data_silos',
    user ='postgres',
    password ='postgres',
    host='localhost'
    )
df_csv = extract_data.extract_csv(csv_file_path)
df_json =  extract_data.extract_json(json_path)

# Consoliudating all the data sources into one common master dataframe 
df_master = extract_data.merge_df([df_sql, df_csv, df_json])

C:\Users\Kartikay\Desktop\Data Engineering\Data-Engineering\DataSilos\extract_data.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query,con=conn1)


 TRANSFORMATION

In [3]:
# Drop unecessary columns
transformation.drop_columns(df_master,['product_id', 'release_date', 'expiry_date', 'weight'])
#Splitting the required columns into sub-columns
transformation.split_columns(df_master, 'product_category', 'product_category', 'product_sub_category', '-', 1)
# Stripping whitespace from string columns
transformation.strip_columns(df_master, ['product_category', 'product_sub_category', 'product_description'])
# Filling missing values in the dataframe
transformation.fill_missing_values(df_master) 

,product_name,product_category,product_price,product_description,manufacturer,stock_quantity,color,master_id,product_sub_category
0,Salsa,Food,631.00,"Fresh and zesty salsa, perfect for nachos.",Meevee,NaN,amber,1,Condiments
1,Party Mini Dress,Clothing,621.39,unknown,Realcube,416.0,None,2,Dresses
2,Cheddar Cheese Crackers,Food,172.69,Crunchy crackers topped with real cheddar chee...,Realpoint,NaN,sapphire,3,Snacks
3,Electric Air Pump,Outdoor,891.12,Fast and convenient air pump for inflating toy...,Voonder,19.0,None,4,unknown
4,Sweet Potato Chips,Food,954.02,"Deliciously crunchy sweet potato chips, season...",Dabfeed,524.0,None,5,Snacks
...,...,...,...,...,...,...,...,...,...
2995,Travel Laundry Bag,Travel,76.05,Compact laundry bag for travel use.,Topicware,NaN,yellow,2996,unknown
2996,Sweetened Condensed Milk,Food,853.48,unknown,Tavu,623.0,yellow,2997,Baking Goods
2997,Saffron Rice Mix,Food,353.72,Flavorful rice mix with saffron and spices.,Jabberstorm,652.0,peach,2998,Grains
2998,Graphic Sweatshirt,Clothing,228.67,A comfortable sweatshirt featuring a bold grap...,Agivu,580.0,None,2999,Tops


In [4]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_name          3000 non-null   object 
 1   product_category      3000 non-null   object 
 2   product_price         3000 non-null   float64
 3   product_description   3000 non-null   object 
 4   manufacturer          2770 non-null   object 
 5   stock_quantity        2252 non-null   float64
 6   color                 2086 non-null   object 
 7   master_id             3000 non-null   int64  
 8   product_sub_category  3000 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 211.1+ KB


VALIDATION

In [5]:
validated_scheme = validation.validate_dataframe(df_master)

LOAD

In [6]:
load.load_data_wrangler(df_master, path="s3://datasilosproj/folder/datasilosproj.parquet")

Success
